# Example to add thermal noise to simulations
This notebook provides an example of how to add thermal noise as different components.\
The identified components are:
* Ice
* Galaxy
* Electronics
  
Each of these components uses a different module from NuRadio

Note that this illustrates the procedure to simulate thermal noise used in the calibration of the absolute system response of the RNO-G experiment.

In [ ]:
import datetime
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import constants

import NuRadioReco
from NuRadioReco.detector.RNO_G.rnog_detector import Detector
from NuRadioReco.framework.event import Event
from NuRadioReco.framework.station import Station
from NuRadioReco.framework.channel import Channel
from NuRadioReco.utilities import units

In [ ]:
logger = logging.getLogger("NuRadioReco")
logger.setLevel(logging.CRITICAL)

We start by generating a set of events with the same structure as simulated events. We do this to use the module.run() convention from NuRadio.

In [ ]:
example_station = 11
channel_id=0

sampling_rate = 3.2 * units.GHz
nr_samples = 2048
frequencies = np.fft.rfftfreq(nr_samples, d=1./sampling_rate)

nr_events = 10

In [ ]:
def generate_event():
    event = Event(run_number=-1, event_id=-1)
    station = Station(example_station)
    station.set_station_time("2023-08-01", format="isot")
    channel_ids = [0]
    for channel_id in channel_ids:
        channel = Channel(channel_id)
        channel.set_frequency_spectrum(np.zeros_like(frequencies, dtype=np.complex128), sampling_rate)
        station.add_channel(channel)
    event.set_station(station)
    return event

ice_events = [generate_event() for e in range(nr_events)]
electronic_events = [generate_event() for e in range(nr_events)]
galactic_events = [generate_event() for e in range(nr_events)]

## Ice noise

The thermal noise generated by the surrounding ice volume is calculated using the channelThermalNoiseAdder module. This module uses pre-generated effective temperature profiles. To create these one can use the code in NuRadioMC/examples/generate/simulate_effective_ice_temperature.

The code propagates rays in the ice starting from the antenna at several incident angles. For each angle, the ice temperature along the ray path is integrated weighed by the local attenuation effects. This yields an effective temperature as seen by the antenna at a specific incident angle.

These profiles are folded into the antenna's vector effective lengths to yield a voltage. Hence the module yields thermal noise from ice <b>before</b> the amplifiers. To obtain the noise at readout one should fold in the system response using the HardwareResponseIncorporator module.

In [ ]:
eff_temp_dir = os.path.join(os.path.dirname(NuRadioReco.__file__), "examples", "RNOG", "eff_temperatures")

In [ ]:
from NuRadioReco.modules.channelThermalNoiseAdder import channelThermalNoiseAdder

detector = Detector(database_connection="RNOG_public", log_level=logging.NOTSET,
                   select_stations=11)
detector_time = datetime.datetime(2023, 8, 1)
detector.update(detector_time)

ice_noise_adder = channelThermalNoiseAdder()
ice_noise_adder.begin(sim_library_dir=eff_temp_dir)
for event in ice_events:
    station = event.get_station()
    ice_noise_adder.run(event, station, detector=detector)

In [ ]:
ice_spectra = []
for event in ice_events:
    station = event.get_station()
    channel = station.get_channel(channel_id)
    spectrum = channel.get_frequency_spectrum()
    ice_spectra.append(np.abs(spectrum))
fig, ax = plt.subplots()
ax.plot(frequencies, np.mean(ice_spectra, axis=0))

## Electronic noise

In principle the electronic noise should be frequency dependent and is ideally derived from a noise temperature measurement. Since these are not available, the electronic noise is approximated as flat with a certain noise temperature. From old measurements (see noise measurements in https://arxiv.org/abs/2411.12922 Figure 8) we approximate this temperature as 80 K.

In the context of the RNO-G system absolute amplitude calibration, an extra free parameter is added to the electronic noise profile to account for this uncertainty. This free parameter module is not included in this example.

In [ ]:
def temp_to_volt(temperature, min_freq, max_freq, frequencies, resistance=50*units.ohm, filter_type="rectangular"):
    if filter_type=="rectangular":
        filt = np.zeros_like(frequencies)
        filt[np.where(np.logical_and(min_freq < frequencies , frequencies < max_freq))] = 1
    else:
        print("Other filters not yet implemented")
    bandwidth = np.trapz(np.abs(filt)**2, frequencies)
    k = constants.k * (units.m**2 * units.kg * units.second**-2 * units.kelvin**-1)
    vrms = np.sqrt(k * temperature * resistance * bandwidth)
    return vrms

In [ ]:
electronic_noise_temp = 80 * units.K
min_freq = 10 * units.MHz
max_freq = 1500 * units.MHz
resistance = 50 * units.ohm

amplitude = temp_to_volt(electronic_noise_temp, min_freq, max_freq, frequencies,
                         resistance, filter_type="rectangular")

In [ ]:
from NuRadioReco.modules.channelGenericNoiseAdder import channelGenericNoiseAdder

electronic_noise_adder = channelGenericNoiseAdder()
electronic_noise_adder.begin()

for event in electronic_events:
    electronic_noise_adder.run(event, station, detector,
                               amplitude=amplitude,
                               min_freq=min_freq, max_freq=max_freq,
                               type="rayleigh")

In [ ]:
electronic_spectra = []
for event in electronic_events:
    station = event.get_station()
    channel = station.get_channel(channel_id)
    spectrum = channel.get_frequency_spectrum()
    electronic_spectra.append(np.abs(spectrum))
fig, ax = plt.subplots()
ax.plot(frequencies, np.mean(electronic_spectra, axis=0))

## Galactic noise

Galactic noise uses the channelGalacticNoiseAdder module. This module reads noise temperatures from a preproduced skymap of the radio galaxy. These signals are propagated through air and ice to the desired antenna location, where the module folds this signal into the antenna vector effectve length.

In [ ]:
from NuRadioReco.modules.channelGalacticNoiseAdder import channelGalacticNoiseAdder

galactic_noise_adder = channelGalacticNoiseAdder()
galactic_noise_adder.begin(freq_range=[min_freq, max_freq],
                          caching=True)

for event in galactic_events:
    galactic_noise_adder.run(event, station, detector)

In [ ]:
galactic_spectra = []
for event in galactic_events:
    station = event.get_station()
    channel = station.get_channel(channel_id)
    spectrum = channel.get_frequency_spectrum()
    galactic_spectra.append(np.abs(spectrum))
fig, ax = plt.subplots()
ax.plot(frequencies, np.mean(galactic_spectra, axis=0))

### Plotting results

In [ ]:
fig, ax = plt.subplots()
ax.plot(frequencies, np.mean(ice_spectra, axis=0), label="Ice", lw=1.)
ax.plot(frequencies, np.mean(electronic_spectra, axis=0), label="Electronic", lw=1.)
ax.plot(frequencies, np.mean(galactic_spectra, axis=0), label="Galactic", lw=1.)
ax.set_xlabel("freq / GHz")
ax.set_ylabel("Spectral amplitude / V/GHz")
ax.legend()
ax.set_title(f"Channel {channel_id}")